### 컬럼설명서

- userid : 회원아이디
- gender : 성별
- membertype_codename : 회원구분 코드의 한글명(학교급) ex. 초등
- grade_codename : 학년 코드
- memberstatus : 회원상태 (정회원, 준회원, 탈퇴회원 등)
- memberstatus_codename : 회원상태 코드의 한글명
- memberstatus_change : 월 중 회원상태 변화
- status_null_count : 회원상태 없음 일수
- statusgroup_10_count : 임시회원 일수
- statusgroup_20_count : 무료회원 일수
- statusgroup_30_count : 유료회원 일수
- statusgroup_40_count : 중지회원 일수
- statusgroup_50_count : 만료회원 일수
- statusgroup_90_count : 해지회원 일수
- point_gain_activeday_count : 포인트 획득일수
- point_gain_count : 포인트 획득 횟수
- point_gain : 획득 포인트
- point_loss_activeday_count : 포인트 차감일수
- point_loss_count : 포인트 차감 횟수
- point_loss : 차감 포인트
- tablet_activeday_count : 기기 활성 횟수
- tablet_moved_menu_count : 기기 메뉴이동 횟수
- tablet_leave_count :기기 물리적 종료 횟수
- tablet_resume_count : 기기 물리적 재개 횟수
- tablet_login_count : 로그인 횟수
- tablet_logout_count : 기기 로그아웃 횟수
- study_activeday_count :학습 활성일 횟수
- study_count : 학습 횟수
- study_notcompleted_count : 학습 미완료 횟수
- study_completed_count : 학습 완료 횟수
- study_restart_count : 학습 재시작 횟수
- total_system_learning_time : 학습 시간합계(시스템)
- total_caliper_learning_time : 학습 시간합계(캘리퍼)
- media_activeday_count : 미디어 활동 활성 일수
- media_count : 미디어 학습 횟수
- video_action_count : 비디오 활동 횟수
- video_start_count : 비디오 시작 횟수
- video_restart_count : 비디오 재시작 횟수
- video_pause_count : 비디오 일시정지 횟수
- video_jump_count : 비디오 점프 횟수
- video_resume_count : 비디오 재개(일시정지 후 횟수)
- video_speed_count : 비디오 속도 조절 횟수
- video_volume_count : 비디오 볼륨 조절 횟수
- video_end_count : 비디오 종료 횟수
- test_activeday_count : 평가 활성 일수
- test_count : 평가 횟수
- test_average_score : 평가 평균 점수
- test_item_count : 평가 문항 개수
- test_correct_count : 평가 정답 개수
- wrong_count : 오답 노트 진입 횟수
- wrong_item_count : 오답 노트 문항 개수
- wrong_correct_count : 오답 노트 정답 개수
- yyyy : 년
- mm : 월

### 데이터 전처리

In [1]:
import pandas as pd

from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import warnings
warnings.filterwarnings('ignore')

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%cd /content/drive/MyDrive/강의자료/머신러닝기초/2일차_분류

/content/drive/MyDrive/강의자료/머신러닝기초/2일차_분류


```python
class0_data = pd.read_csv('./datasets/만료및탈퇴회원.csv', index_col=0)
class1_data = pd.read_csv('./datasets/정회원.csv', index_col=0)
display(class0_data, class1_data)
```

**분석 목표 : 다음달에 탈퇴하는 회원의 현재달의 징조 파악하기**

```python
## 11월에 탈퇴한 회원의 10월 데이터 - 탈퇴회원 데이터
exit_user = class0_data[class0_data['mm'] == 11]['userid'].unique()
exit_data = class1_data[(class1_data['userid'].isin(exit_user)) & (class1_data['mm'] == 10)]

exit_data['target'] = 0

exit_data

```

```python
## 11월에 정회원인 회원의 10월 데이터 - 정회원 데이터
regular_user = class1_data[class1_data['mm'] == 11]['userid'].unique()
regular_data = class1_data[(class1_data['userid'].isin(regular_user)) & (class1_data['mm'] == 10)]

regular_data['target'] = 1

regular_data
```

```python
dataset = pd.concat([exit_data, regular_data])
```

```python
dataset['target'].value_counts()
```

```python
dataset = dataset.dropna(subset = ['point_gain_activeday_count', 'point_gain_count', 'point_gain',
             'point_loss_activeday_count', 'point_loss_count', 'point_loss',
             'tablet_activeday_count', 'tablet_moved_menu_count', 'tablet_leave_count', 'tablet_resume_count',
             'tablet_login_count', 'tablet_logout_count', 'study_activeday_count', 'study_count',
             'study_notcompleted_count', 'study_completed_count',
             'study_restart_count', 'total_system_learning_time', 'total_caliper_learning_time',
             'media_activeday_count', 'media_count',
             'video_action_count', 'video_start_count', 'video_restart_count',
             'video_pause_count', 'video_jump_count', 'video_resume_count',
             'video_speed_count', 'video_volume_count', 'video_end_count',
             'test_activeday_count', 'test_count', 'test_average_score',
             'test_item_count', 'test_correct_count', 'wrong_count',
             'wrong_item_count', 'wrong_correct_count'
             ])
```

```python
X = dataset[['point_gain_activeday_count', 'point_gain_count', 'point_gain',
             'point_loss_activeday_count', 'point_loss_count', 'point_loss',
             'tablet_activeday_count', 'tablet_moved_menu_count', 'tablet_leave_count', 'tablet_resume_count',
             'tablet_login_count', 'tablet_logout_count', 'study_activeday_count', 'study_count',
             'study_notcompleted_count', 'study_completed_count',
             'study_restart_count', 'total_system_learning_time', 'total_caliper_learning_time',
             'media_activeday_count', 'media_count',
             'video_action_count', 'video_start_count', 'video_restart_count',
             'video_pause_count', 'video_jump_count', 'video_resume_count',
             'video_speed_count', 'video_volume_count', 'video_end_count',
             'test_activeday_count', 'test_count', 'test_average_score',
             'test_item_count', 'test_correct_count', 'wrong_count',
             'wrong_item_count', 'wrong_correct_count'
             ]]
```

```python
Y = dataset['target']
```

```python
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)
```

### 모델 적용

#### Bagging(배깅)

```python
from sklearn.ensemble import BaggingClassifier
from sklearn.metrics import accuracy_score, recall_score, precision_score, roc_auc_score, confusion_matrix
lr_clf = LogisticRegression(solver='liblinear')

bagging_clf = BaggingClassifier(base_estimator=lr_clf)

# BaggingClassifier 학습/예측.
bagging_clf.fit(X_train , y_train)
pred = bagging_clf.predict(X_test)
```

```python
# 평가
accuracy = accuracy_score(y_test , pred)
recall = recall_score(y_test, pred)
precision = precision_score(y_test, pred)
auc = roc_auc_score(y_test, pred)
matrix = confusion_matrix(y_test, pred)

print('Bagging 분류기 정확도: {0:.4f}'.format(accuracy))
print('Bagging 분류기 Recall: {0:.4f}'.format(recall))
print('Bagging 분류기 Precision: {0:.4f}'.format(precision))
print('Bagging 분류기 AUC: {0:.4f}'.format(auc))
print('Bagging 분류기 Confusion Matrix:','\n', matrix)
```

```python
# 개별 모델의 학습/예측/평가.
lr_clf.fit(X_train , y_train)
pred = lr_clf.predict(X_test)
class_name= lr_clf.__class__.__name__

accuracy = accuracy_score(y_test , pred)
recall = recall_score(y_test, pred)
precision = precision_score(y_test, pred)
auc = roc_auc_score(y_test, pred)
matrix = confusion_matrix(y_test, pred)

print('{0} 정확도: {1:.4f}'.format(class_name, accuracy))
print('{0} Recall: {1:.4f}'.format(class_name, recall))
print('{0} Precision: {1:.4f}'.format(class_name, precision))
print('{0} AUC: {1:.4f}'.format(class_name, auc))
print('{0} Confusion Matrix:'.format(class_name),'\n', matrix)
```


#### 랜덤 포레스트

```python
from sklearn.ensemble import RandomForestClassifier

rf_clf = RandomForestClassifier(n_estimators=100, random_state=0, max_depth=8)
rf_clf.fit(X_train , y_train)
pred = rf_clf.predict(X_test)

accuracy = accuracy_score(y_test , pred)
recall = recall_score(y_test, pred)
precision = precision_score(y_test, pred)
auc = roc_auc_score(y_test, pred)
matrix = confusion_matrix(y_test, pred)

print('랜덤 포레스트 정확도: {0:.4f}'.format(accuracy))
print('랜덤 포레스트 Recall: {0:.4f}'.format(recall))
print('랜덤 포레스트 Precision: {0:.4f}'.format(precision))
print('랜덤 포레스트 AUC: {0:.4f}'.format(auc))
print('랜덤 포레스트 Confusion Matrix:','\n', matrix)
```

```python
ftr_importances_values = rf_clf.feature_importances_
ftr_importances = pd.Series(ftr_importances_values,index=X_train.columns)
ftr_importances.sort_values(ascending=False)[:20]
```

```python
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

ftr_importances_values = rf_clf.feature_importances_
ftr_importances = pd.Series(ftr_importances_values,index=X_train.columns  )
ftr_top20 = ftr_importances.sort_values(ascending=False)[:20]

plt.figure(figsize=(8,6))
plt.title('Feature importances Top 20')
sns.barplot(x=ftr_top20 , y = ftr_top20.index)
plt.show()
```

#### 그래디언트 부스팅

```python
from sklearn.ensemble import GradientBoostingClassifier
import time
import warnings
warnings.filterwarnings('ignore')

# GBM 수행 시간 측정을 위함. 시작 시간 설정.
start_time = time.time()

gb_clf = GradientBoostingClassifier(random_state=0)
gb_clf.fit(X_train , y_train)
gb_pred = gb_clf.predict(X_test)

accuracy = accuracy_score(y_test , gb_pred)
recall = recall_score(y_test, gb_pred)
precision = precision_score(y_test, gb_pred)
auc = roc_auc_score(y_test, gb_pred)
matrix = confusion_matrix(y_test, gb_pred)

print('GBM 정확도: {0:.4f}'.format(accuracy))
print('GBM Recall: {0:.4f}'.format(recall))
print('GBM Precision: {0:.4f}'.format(precision))
print('GBM AUC: {0:.4f}'.format(auc))
print('GBM Confusion Matrix:','\n', matrix)

print("GBM 수행 시간: {0:.1f} 초 ".format(time.time() - start_time))
```